# This notebook gathers data of NISA league stats tables

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Optional cell, stretches cell width for better readability
from IPython.core.display import HTML

custom_css = """
<style>
.container { width: 100% !important; }
.code_cell { flex-grow: 1; width: 100% !important; }
.code_cell .input_area { width: 100% !important; }
</style>
"""

display(HTML(custom_css))

In [3]:
def extract_stats_table(soup):
    
    table = soup.find('table')
    
     # Extract the table data into a list of dictionaries
    data = []
    rows = table.find_all('tr')
    headers = [header.get_text(strip=True) for header in rows[0].find_all('th')]
    
    for row in rows[1:]:
        values = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]
        data.append(dict(zip(headers, values)))

    # Creating DataFrame
    league_stats_df = pd.DataFrame(data)

    return(league_stats_df)

In [4]:
def create_df(csv_filename):
    
    soup = BS(driver.page_source,'html.parser')
    
    league_stats_df = extract_stats_table(soup)
    
    # Creates a "data" folder if it doesn't exist
    data_folder = 'data'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    #Saves the dataframe locally to a csv
    csv_filepath = os.path.join(data_folder, csv_filename)
    league_stats_df.to_csv(csv_filepath, index=False)
    
    print(league_stats_df)

In [5]:
def download_table(desired_table, desired_year, file_name):
    
    # Find and click the button to open the drop-down for selecting the year
    year_button = driver.find_element(By.CSS_SELECTOR, 'span#select2-year-container')
    year_button.click()

    # Find and click the specific year option in the drop-down
    year_option = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, f'//li[text()="{desired_year}"]')))
    year_option.click()

    # Find and click the button to open the drop-down for selecting the table
    table_button = driver.find_element(By.CSS_SELECTOR, 'span#select2-type-container')
    table_button.click()

    # Find and click the specific table option in the drop-down
    table_option = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, f'//li[text()="{desired_table}"]')))
    table_option.click()

    # Call create_df function to download the table data
    create_df(f"{file_name}_{desired_year}.csv")

    driver.quit()

In [6]:
def print_options_for_year(desired_year):
    
    driver = webdriver.Chrome()

    # Navigating to the page
    URL = 'https://nisaofficial.com/league-stats'
    driver.get(URL)

    # Find and click the button to open the drop-down for selecting the year
    year_button = driver.find_element(By.CSS_SELECTOR, 'span#select2-year-container')
    year_button.click()

    # Find and click the specific year option in the drop-down
    year_option = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, f'//li[text()="{desired_year}"]')))
    year_option.click()

    # Wait for the third button to be clickable
    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span#select2-type-container')))

    # Hover over the third button to trigger its drop-down
    third_button = driver.find_element(By.CSS_SELECTOR, 'span#select2-type-container')
    driver.execute_script("arguments[0].click();", third_button)

    # Wait for the drop-down to open
    table_dropdown = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'select#type')))

    # Find the drop-down options and print their text
    options = table_dropdown.find_elements(By.TAG_NAME, 'option')
    for option in options:
        print(option.text)
    
    driver.quit()

## Run the below cell to see all the available years.

In [7]:
# Start Chrome with Selenium
driver = webdriver.Chrome()

# Navigate to the page with player stats
player_stats_url = 'https://nisaofficial.com/league-stats'
driver.get(player_stats_url)

# Interaction with the first button (year selection)
year_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 'select2-year-container')))
year_button.click()

# Get the available year options from the drop-down menu
year_options = driver.find_elements(By.CSS_SELECTOR, 'li.select2-results__option')
years = [option.text for option in year_options]
print(years)

# Quit the browser
driver.quit()

['2019-2020', '2020-2021', '2021', '2022', '2023']


## Run the below cell to see the filters you can chose based on the year you're interested in.

In [8]:
print_options_for_year('2023')

Choose Type
Championship
Exhibition
NISA Challenge Cup
Playoffs
Regular Season
US Open Cup


## Table Selection
### Run both cells for every table you want to download. All you need to do is change are the arguements in the 'download_table()' function. The first arguement is the type of table you want to download (optional arguements come from the 'print_options_for_year()' function. The second arguement is what year you want to pull a table from, and the third is what you want the file to be called. You do not need to end the file name with '.csv'. The file will be downloaded into a directory called 'data'. You do not have to create this directory yourself. The 'create_df()' function will do this for you. Finally, make sure your agruements are encased in apostrophes or quotations.

In [9]:
# Start Chrome with Selenium
driver = webdriver.Chrome()

# Navigate to the page
URL = 'https://nisaofficial.com/league-stats'
driver.get(URL)

In [10]:
download_table('Regular Season', '2023', 'regseason_leaguestat')

                     TEAM  GP GOALS SHOTS YEL RED
0          Chattanooga FC  16    30   105  29   0
1     Maryland Bobcats FC  18    24    89  28   1
2        ALBION San Diego  17    19    91  36   4
3         Club de Lyon FC  16    19    86  59   1
4       Los Angeles Force  15    19    78  32   3
5       Michigan Stars FC  15    18    60  35   4
6       Flower City Union  17    17    78  41   2
7    Gold Star FC Detroit  14    16    68  38   2
8     Savannah Clovers FC  16     9    62  37   2


## Use the below cell to download all Regular Season Tables
### You have to choose 'Choose Type' to get the regular season stats of years besides 2023. This is the only way to refresh the table. If you have the program click on 'Regular Season' it will not refresh the table and will keep giving you the data from 2023.

In [11]:
# Loop through the years and download tables
for year in years:
    
    print(f'\nLeague Stats for {year}')
    
    # Start Chrome with Selenium
    driver = webdriver.Chrome()

    # Navigate to the page
    URL = 'https://nisaofficial.com/league-stats'
    driver.get(URL)
    
    download_table('Choose Type', year, 'regseason_leaguestat')


League Stats for 2019-2020
                       TEAM GP GOALS SHOTS YEL RED
0                  Miami FC  7    22    41   5   1
1    Cal United Strikers FC  7    17    34  10   0
2              NISA Atlanta  6    13    15   9   0
3        Stumptown Athletic  6    13     5   6   0
4          Oakland Roots SC  6    10    19  15   1
5          ALBION San Diego  5     9    49  11   1
6         Los Angeles Force  5     6    12  17   0
7         Philadelphia Fury  6     1     8   2   0

League Stats for 2020-2021
                         TEAM GP GOALS SHOTS YEL RED
0              Chattanooga FC  3     9     0   0   0
1          Lone Star Republic  3     9     0   0   0
2         Maryland Bobcats FC  3     9     0   0   0
3             Detroit City FC  2     6     0   0   0
4       FC Baltimore Christos  3     6     0   0   0
5                    Gaffa FC  3     4     0   0   0
6                Soda City FC  3     4     0   0   0
7                Cleveland SC  2     3     0   0   0
8       

## Below will concat all the tables into one given you used the same file naming.

In [12]:
data_frames = []

# Read and preprocess data for each year
for year in years:
    filename = f'data/regseason_leaguestat_{year}.csv'
    df = pd.read_csv(filename)
    
    # Remove the first column
    df = df.iloc[:, 1:]
    
    # Add a column for the year
    df['Year'] = year
    
    # Append the modified DataFrame to the list
    data_frames.append(df)

# Concatenate all DataFrames into a single DataFrame
complete_regseason_leaguestat_df = pd.concat(data_frames, ignore_index=True)

# Save the concatenated DataFrame to a CSV file
complete_regseason_leaguestat_df.to_csv('complete_regseason_leaguestat.csv', index=False)

print(complete_regseason_leaguestat_df)

                      TEAM  GP  GOALS  SHOTS  YEL  RED       Year
0                 Miami FC   7     22     41    5    1  2019-2020
1   Cal United Strikers FC   7     17     34   10    0  2019-2020
2             NISA Atlanta   6     13     15    9    0  2019-2020
3       Stumptown Athletic   6     13      5    6    0  2019-2020
4         Oakland Roots SC   6     10     19   15    1  2019-2020
..                     ...  ..    ...    ...  ...  ...        ...
67       Los Angeles Force  15     19     78   32    3       2023
68       Michigan Stars FC  15     18     60   35    4       2023
69       Flower City Union  17     17     78   41    2       2023
70    Gold Star FC Detroit  14     16     68   38    2       2023
71     Savannah Clovers FC  16      9     62   37    2       2023

[72 rows x 7 columns]


In [13]:
complete_regseason_leaguestat_df.to_csv('data/complete_regseason_leaguestat_df.csv')